In [1]:
import datetime
import QuantLib as ql
from module.Swap_Curve import GET_QUOTE, SWAP_CURVE # 이전에 만든 함수들 가져오기

In [2]:
class FRA():
    # __init__는 class 초기화(내부화) 기능(class 내부에서 변수를 처리할 수 있도록 만듬?)
    def __init__(self, todays_date, effective_date, maturity_date, position, fra_rate, notional): # 입력 변수 6개
        
        # Initial Setup 1 : Date & Curve
        self.date = todays_date # class 내부에서 self.date는 todays_date로 지정함
        self.curve = self.CURVE(self.date) # class 내부에서 self.curve는 todays_date에 커브의 값을 지정함
        
        # Initial Setup 2 : Instruments Info
        self.effective_date = ql.Date(effective_date.day, effective_date.month, effective_date.year) # 퀀트립 형태의 날짜로 바꿈
        self.maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year) # 퀀트립 형태의 날짜로 바꿈
        if position == 'Long':
            self.position = ql.Position.Long # 선도 금리 계약의 포지션을 Long으로 지정
        else:
            self.position = ql.Position.Short # 선도 금리 계약의 포지션을 Short로 지정
        self.fra_rate = fra_rate
        self.notional = notional

        # Pricing Results(나중에 구현할 함수를 미리 지정해놓음)
        self.npv = self.PRICING(self.curve) # 이론가 계산할 수 있는 함수
        self.dv01 = self.DV01() # 금리 커브 변화에 따른 포지션의 변화를 계산할 수 있는 델타 함수
        self.theta = self.THETA() # 시장 환경은 변화하지 않았지만, 시간이 변할 때 포지션 변화를 계산할 수 있는 세타 함수
        
    def CURVE(self, date): # class 내부의 함수들은 모두 self를 써줌
        return SWAP_CURVE(date, GET_QUOTE(date))
    
    def PRICING(self, curve):
        curve_handle = ql.YieldTermStructureHandle(curve) # 커브 핸들 변수 생성
        libor = ql.USDLibor(ql.Period(3, ql.Months), curve_handle) # 준거 금리 변수 생성(여기서는 3개월로 지정함)
        # Pricing FRA
        fra = ql.ForwardRateAgreement(self.effective_date, # 정산일
                                      self.maturity_date, # 만기일자
                                      self.position, # 포지션(매매방향)
                                      self.fra_rate, # 계약 금리
                                      self.notional, # 명목 금액
                                      libor, # 준거 금리(3개월)
                                      curve_handle) # 평가 금리 커브(그냥 curve가 아닌 curve_handle 넣어야함)
        npv = fra.NPV() # NPV() ==> fra의 이론적인 가치를 구하고 싶을 때
        return npv
    
    def DV01(self): # 금리 커브가 1bp 움직였을 때, 선도 금리 계약(FRA)의 가격이 얼마나 변하는지
        curve_handle = ql.YieldTermStructureHandle(self.curve) # 커브 핸들 변수 생성(PRICING과 같음)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(basis_point)))
        # ZeroSpreadedTermStructure ==> 기존의 기간 구조를 받은 후 몇 bp 위 or 아래로 움직여줄 것인지를 구현하는 함수
        up_fra = self.PRICING(up_curve) # +1bp 움직였을 때, Pricing

        # FRA price when 1bp down
        down_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(-basis_point)))
        down_fra = self.PRICING(down_curve) # -1bp 움직였을 때, Pricing

        # Delta 계산
        dv01 = (up_fra - down_fra) / 2
        return dv01
    
    def THETA(self): # 오직 시간이 한 단위 변했을 때(시장 환경엔 변화 X), 선도 금리 계약의 가격이 얼마나 변하는지
        price_t0 = self.PRICING(self.CURVE(self.date)) # 기존 pricing
        print(price_t0)
        
        price_t1 = self.PRICING(self.CURVE(self.date + datetime.timedelta(days=1))) # 하루 지난 후 pricing
        print(price_t1)
        return price_t1 - price_t0 # THETA 계산

In [3]:
if __name__ == "__main__":

    # 오늘 날짜를 날짜 형식으로 변환
    todays_date = datetime.date(2020, 10, 9)
            
    # FRA의 상품 정보
    effective_date = datetime.date(2020, 10, 19) # 정산일
    maturity_date = datetime.date(2020, 12, 15) # 만기일
    position = 'Long' # 포지션 매수(Long)
    fra_rate = 0.0022 # 선도 금리 수준
    notional = 10000000 # USD 달러 기준 1,000만 불
    
    # Build FRA object
    fra = FRA(todays_date, # 오늘 날짜
              effective_date, # 정산일
              maturity_date, # 만기일
              position, # 포지션
              fra_rate, # 선도 금리 수준
              notional)
    
    print("Price = {}".format(round(fra.npv, 4)))
    print("DV01 = {}".format(round(fra.dv01, 4)))
    print("Theta = {}".format(round(fra.theta, 4)))

84.6341973176786
84.64588629561904
Price = 84.6342
DV01 = 158.3565
Theta = 0.0117
